# Limpieza de datos
 
La limpieza de datos es uno de los pasos más importantes en un proyecto de aprendizaje automático, pues, pues el modelo usará las propiedades de cada variable, y si se entrena con datos "sucios" probablemente la salida también esté "sucia".
 
Para evaluar la validez de los datos de nuestro dataset, es importante analizar y corroborar si los datos cumplen o se ajustan a la reglas o restricciones propias del dato:
* **De tipo de dato**: los valores en una columna en particular deben ser de un tipo de datos.
* **De rango**: generalmente, los números o fechas deben estar dentro de un cierto rango.
* **Obligatorias**: determinadas columnas no pueden estar vacías.
* **Únicas**: un campo, debe ser único en un conjunto de datos.
* **De pertenencia al conjunto**: los valores de una columna provienen de un conjunto de valores discretos. Por ejemplo, el sexo biológico de una persona en general se marca como masculino o femenino.
* **Patrones de expresión regular**: campos de texto que deben seguir un patrón determinado. (Email)
* **Validación de campo cruzado**: deben cumplirse determinadas condiciones que abarcan varios campos. Por ejemplo, la fecha de alta de un paciente del hospital no puede ser anterior a la fecha de admisión.
 

El encuentro pasado hicimos una revisión de las columnas y descubrimos que había datos nulos (registros vacíos). No es posible realizar un modelo de aprendizaje automático con datos nulos por lo que veremos diferentes maneras de resolver esta cuestión.

Hay tres posiblidades frente a datos nulos: **imputar**, **marcar** o **eliminar**. En esta Notebook veremos algunos métodos para realizarlo.

In [2]:
#importamos las librerias que utilizaremos

import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns

#### Desde el Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Recordar utilizar dataset modificado de la clase pasada. 

data = pd.read_csv("/content/drive/MyDrive/Aprender Programando/2022/Guías y Recursos - Trayectos 2021/Recursos/Modulos 1 y 2. Presentación CABA/Ciencia de datos/2022/Módulo 2/Encuentro 2/data_clima.csv")

#### Desde el archivo descargado en la computadora

In [ ]:
from google.colab import files
import io

filesUploaded = files.upload()

In [3]:
data = pd.read_csv(".csv")

In [4]:
#vemos los primeros registros del dataset

data.head(3)

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
0,0.92,11.27,130.0,8.05,1021.60,no,Cold,-0.56
1,0.73,20.93,330.0,16.10,1017.00,si,Warm,21.11
2,0.97,5.97,193.0,14.91,1013.99,si,Normal,16.60


In [5]:
# Vemos el tamaño del dataset

data.shape

(10000, 8)

#### Identificación de datos nulos

Para observar si hay datos nulos se puede utilizar el método *isnull()* o *isna()* el cual devovlerá **True** en el caso que haya un dato nulo y **False** en el caso que no lo haya.
Esto puede ser utilizado como una máscara para filtrar el dataset y ver los registros nulos.

Para saber la cantidad de datos nulos es posible utilizar *sum()* luego de la identificación de los nulos sabiendo que **True** cuenta como 1 y **False** cuenta como 0, de esta manera sumara 1 por cada dato nulo. 

In [6]:
data.isnull()

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
9995,False,False,False,False,False,False,False,False
9996,False,False,False,False,False,False,False,False
9997,False,False,False,False,False,False,False,False
9998,False,False,False,False,False,False,False,False


In [7]:
data.isna()

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
9995,False,False,False,False,False,False,False,False
9996,False,False,False,False,False,False,False,False
9997,False,False,False,False,False,False,False,False
9998,False,False,False,False,False,False,False,False


In [8]:
data.isnull().sum()

humedad                 5
velocidad_viento_kmh    5
rumbo_viento_grados     5
visibilidad_km          5
presion_mbar            9
lluvia                  1
descripcion             8
temperatura             0
dtype: int64

Podemos observar que *temperatura* que es nuestra variable a predecir no tiene ningún nulo y hay una pequeña cantidad en cada una de las otras columnas. Exploraremos cada caso para definir si **imputar**, **marcar** o **eliminar** los nulos y formas de realizarlo.

#### Visiblidad

Comenzaremos observando la columna *visibilidad*, cuáles son los datos nulos. Para esto realizaremos un filtro utilizando el método *isnull()* para crear una máscara y luego filtraremos con *.loc.* para observar los datos

In [9]:
nulo_visibilidad = data["visibilidad_km"].isnull()
nulo_visibilidad

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Name: visibilidad_km, Length: 10000, dtype: bool

In [10]:
data.loc[nulo_visibilidad]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
568,NaN,NaN,NaN,NaN,NaN,si,Normal,16.07
2548,NaN,NaN,NaN,NaN,NaN,si,Warm,27.76
3545,NaN,NaN,NaN,NaN,NaN,si,Normal,17.74
6580,NaN,NaN,NaN,NaN,NaN,si,Warm,22.97
7815,NaN,NaN,NaN,NaN,NaN,si,NaN,0.98


En este caso observamos que las 5 filas tienen varios datos faltantes de muchas columnas, imputar en este caso tal vez sería un poco forzado ya que de esos registros faltan muchos datos.

En este caso lo que realizaremos es **eliminar** las filas que tienen muchos datos nulos utilizando el método [dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html). Este método permite eliminar filas o columnas y se pueden definir distintos criterios. En este caso eliminaremos los 5 registos que tienen 5 datos nulos.

In [11]:
# qué datos toma la variable

data.dropna(axis=0, thresh=5, inplace=True)

In [12]:
# Revisamos la operación

data.loc[nulo_visibilidad]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura


In [14]:
# Comprobamos el tamaño del dataset y la cantidad de nulos

print(data.shape)
data.isnull().sum()

(9995, 8)


humedad                 0
velocidad_viento_kmh    0
rumbo_viento_grados     0
visibilidad_km          0
presion_mbar            4
lluvia                  1
descripcion             7
temperatura             0
dtype: int64

#### Lluvia

Revisaremos el caso de la columna *lluvia* observando cómo es el dato nulo. Nuevamente utilizaremos el método *isnull()* como una máscara y luego el método *.loc* para observar el dato

In [15]:
nulo_lluvia = data["lluvia"].isnull()
nulo_lluvia

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Name: lluvia, Length: 9995, dtype: bool

In [16]:
data.loc[nulo_lluvia]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
1862,0.97,9.58,299.0,11.27,1003.99,NaN,Normal,17.64


Podemos observar que es solamente un dato y no se observa ninguna otra particularidad ya que el resto de los valores parecen correctos.

En este caso lo que realizaremos es **imputar** de manera aleatoria el dato faltante. Al ser una variable categórica utilizaremos la **moda**, es decir, el valor que más veces se repite. Para esto volveremos a observar como estás distribuidos los valores que puede tomar la columna y luego definiremos con *.loc* el dato faltante y lo completaremos.

En este caso lo realizaremos de manera aleatoria utilizando como criterio la **moda**, también podría imputarse los datos faltantes en base a los otros datos de las columnas que si tenemos, por ejemplo, *humedad* o *descripcion*.

In [17]:
# qué datos toma la variable

data["lluvia"].value_counts()

si    8906
no    1088
Name: lluvia, dtype: int64

In [19]:
# Como la moda es "si" (el valor que más se repite) imputaremos con ese valor (que además corresponde con la humedad)

# Seleccionaremos el dato que queremos imputar

data.loc[nulo_lluvia, "lluvia"]

1862    NaN
Name: lluvia, dtype: object

In [20]:
# Imputaremos el dato con el valor que definamos, en este caso "si"

data.loc[nulo_lluvia, "lluvia"] = "si"

In [21]:
# Revisamos la operación

data.loc[nulo_lluvia]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
1862,0.97,9.58,299.0,11.27,1003.99,si,Normal,17.64


In [22]:
# Comprobamos que este correctamente imputado y que no haya más datos nulos en lluvia

print(data["lluvia"].value_counts())
data.isnull().sum()

si    8907
no    1088
Name: lluvia, dtype: int64


humedad                 0
velocidad_viento_kmh    0
rumbo_viento_grados     0
visibilidad_km          0
presion_mbar            4
lluvia                  0
descripcion             7
temperatura             0
dtype: int64

#### Descripción

Ahora realizaremos el mismo procedimiento en el caso de los datos nulos en la variable *decripción*, utilizaremos *isnull()* para crear una máscara y luego filtraremos con *.loc.* para observar los datos

In [23]:
nulo_descripcion = data["descripcion"].isnull()
nulo_descripcion

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Name: descripcion, Length: 9995, dtype: bool

In [24]:
data.loc[nulo_descripcion]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
549,0.55,20.46,160.0,10.21,1020.71,si,NaN,8.91
1485,0.56,16.04,33.0,10.35,1021.43,si,NaN,2.83
2224,0.48,11.27,120.0,9.98,1011.10,si,NaN,16.11
3485,0.45,13.78,341.0,16.10,1019.04,si,NaN,23.79
5382,0.92,4.73,170.0,3.53,1006.13,no,NaN,-5.16
6582,0.65,27.10,319.0,9.98,0.00,si,NaN,4.95
9614,0.82,11.27,10.0,16.10,1019.90,si,NaN,0.56


Podemos observar que son 7 datos que únicamente es nulo el dato "descripcion" y que el resto de los valores parece razonable. 

En este caso nuevamente lo que realizaremos es **imputar** de manera aleatoria el dato faltante. Para esto volveremos a observar los datos que puede tomar esta columna y luego definiremos con *.loc* el dato faltante y lo completaremos.Esta imputación será creando una lista aleatoria de valores para completar los datos faltantes. 

Para crear esta lista utilizaremos la libreria *random* cuyo método *choices* permite la generación de una lista aleatoria. Para ver más de esta librería y practicar puede consultaste [w3schools](https://www.w3schools.com/python/ref_random_choice.asp)

Aquí también lo realizaremos de manera aleatoria, pero podría imputarse los datos faltantes en base a los otros datos de las columnas que si tenemos.

In [25]:
# Primero confirmaremos los valores que puede tomar la columna "descripcion"

data["descripcion"].unique()

array(['Cold', 'Warm', 'Normal', nan], dtype=object)

In [28]:
# Creamos la lista que tome los valores que indiquemos y k es la cantidad de valores que se generaran, en este caso 7

import random

lista_imputar = random.choices(["Cold", "Warm", "Normal"], k=7)  #correr varias veces para ver como se modifica
lista_imputar

['Cold', 'Warm', 'Normal', 'Normal', 'Normal', 'Cold', 'Normal']

In [29]:
# Volvemos a seleccionar los campos que queremos imputar y definimos los campos con la lista creada en el paso anterior

data.loc[nulo_descripcion, "descripcion"]=lista_imputar

In [30]:
# Comprobamos que este correctamente imputado y que no haya más datos nulos en lluvia

print(data["descripcion"].value_counts())

data.isnull().sum()

Normal    4991
Warm      2505
Cold      2499
Name: descripcion, dtype: int64


humedad                 0
velocidad_viento_kmh    0
rumbo_viento_grados     0
visibilidad_km          0
presion_mbar            4
lluvia                  0
descripcion             0
temperatura             0
dtype: int64

#### Presion mbar

Ahora realizaremos el mismo procedimiento en el caso de los datos nulos en la variable "presion", utilizaremos *isnull()* para crear una máscara y luego filtraremos con *.loc.* para observar los datos

In [31]:
# En la misma linea

data.loc[data["presion_mbar"].isnull()]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
1253,0.42,9.77,138.0,10.35,NaN,si,Normal,13.98
4135,0.48,25.89,302.0,10.58,NaN,si,Normal,17.00
5352,0.92,9.60,154.0,14.59,NaN,si,Cold,3.51
8128,0.35,4.80,50.0,9.98,NaN,si,Warm,27.20


Aquí se observan 4 datos nulos pero nuevamente no se observan particularidades en el resto de los datos. Debido a esto también vamos a **imputar** los datos faltantes , en este caso para imputarlo utilizaremos el método [fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html?highlight=fillna#pandas.DataFrame.fillna). Este método imputa valores nulos con el valor que se desee, en este caso imputaremos con la **media (promedio)** de la columna

In [32]:
# calculamos la media (promedio)

promedio_presion = data["presion_mbar"].mean()

promedio_presion

1004.8844069662696

In [33]:
# se puede redondear el valor utilizando round

promedio_presion_redondeado = round(promedio_presion,2)
promedio_presion_redondeado

1004.88

In [34]:
data["presion_mbar"] = data["presion_mbar"].fillna(promedio_presion_redondeado)

In [35]:
# Revisamos la operación

data.loc[data["presion_mbar"].isnull()]

,humedad,velocidad_viento_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura


In [36]:
# Comprobamos la cantidad de nulos

data.isnull().sum()

humedad                 0
velocidad_viento_kmh    0
rumbo_viento_grados     0
visibilidad_km          0
presion_mbar            0
lluvia                  0
descripcion             0
temperatura             0
dtype: int64

## Conclusión

En esta notebook hemos revisado disntias formas de resolver los datos faltantes y se vieron diferentes métodos útiles para realizar estas operaciones. 

Dado que hemos realizado cambios en el Dataset y el encuentro que viene retomaremos este mismo, vamos a utilizar el método *to_csv* que es el opuesto a *read_csv* y sirve para exportar un CSV de una Notebook

In [ ]:
# Guardar en drive

data.to_csv("/content/drive/MyDrive/Aprender Programando/2022/Guías y Recursos - Trayectos 2021/Recursos/Modulos 1 y 2. Presentación CABA/Ciencia de datos/2022/Módulo 2/Encuentro 3/data_clima", index=False)


NameError: ignored

In [ ]:
# Descargar en computadora

from google.colab import files

data.to_csv("data_clima3.csv", index=False)
files.download("data_clima3.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>